In [13]:
from pyspark.sql import SparkSession
import boto3
from dotenv import load_dotenv
import os

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter as chaves da AWS das variáveis de ambiente
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# Iniciando uma SparkSession com Delta Lake
spark = SparkSession.builder \
    .appName("DeltaLakeSparkS3") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Adicionando configurações S3 diretamente na sessão Spark
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key_id)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_access_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Configurações dos buckets S3
landing_bucket = "engenharia-dados-satc-landing-zone-bucket"
bronze_bucket = "engenharia-dados-satc-bronze-bucket"

# Verificando os arquivos no bucket S3
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
response = s3.list_objects_v2(Bucket=landing_bucket)

print("Arquivos no bucket S3:")
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("Nenhum arquivo encontrado no bucket especificado.")

# Função para ler dados do S3
def read_from_s3(bucket, path):
    try:
        df = spark.read.format("csv").option("header", "true").load(f"s3a://{bucket}/{path}")
        return df
    except Exception as e:
        print(f"Erro ao ler dados do S3: {e}")

# Função principal
def main():
    try:
        # Definindo os caminhos dos arquivos CSV na landing
        paths = ['geo.csv', 'dadosprodutos.csv', 'pedidos.csv', 'clientes.csv', 'vendedores.csv', 'pagamentos.csv', 'dadospedidos.csv']
        dfs = {}
        for path in paths:
            df_name = path.split('.')[0]
            dfs[df_name] = read_from_s3(landing_bucket, f"/{path}")

        # Salvando os DataFrames no formato Parquet usando Delta Lake no bucket da camada bronze
        for df_name, df in dfs.items():
            df.write.format("delta").mode('overwrite').save(f"s3a://{bronze_bucket}/{df_name}")
        
        print("Dados salvos na camada bronze com sucesso.")
        
    except Exception as e:
        print(f"Erro no processo principal: {e}")

if __name__ == "__main__":
    main()
